In [ ]:
import pyodbc
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Chuỗi kết nối SQL Server
connection_string = (
    "DRIVER={SQL Server};"
    "SERVER=LAPTOP-IP1RMDOK\\DATASERVERNGHIA;"
    "DATABASE=restaurants_old;"
    "UID=sa;"
    "PWD=123456;"
    "Trusted_Connection=no;"  # Đảm bảo sử dụng xác thực SQL
)


# Hàm thiết lập cơ sở dữ liệu với kích thước cột phù hợp
def setup_database():
    """Thiết lập kết nối cơ sở dữ liệu và tạo bảng với kích thước cột phù hợp cho URL"""
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Tạo bảng restaurants
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'restaurants')
        CREATE TABLE restaurants (
            restaurant_id INT IDENTITY(1,1) PRIMARY KEY,
            url NVARCHAR(2000),
            restaurant_name NVARCHAR(500),
            restaurant_type NVARCHAR(500),
            rating FLOAT,
            phone NVARCHAR(20),
            price_level NVARCHAR(50),
            address NVARCHAR(1000),
            opening_hours NVARCHAR(MAX),
            created_at DATETIME DEFAULT GETDATE()
        )
        ''')

        # Tạo bảng reviews
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'reviews')
        CREATE TABLE reviews (
            id INT IDENTITY(1,1) PRIMARY KEY,
            restaurant_id INT,
            reviewer_name NVARCHAR(500),  
            reviewer_info NVARCHAR(500),  
            rating FLOAT,
            review_time NVARCHAR(100),
            review_text NVARCHAR(MAX),
            service_rating NVARCHAR(50),
            food_rating NVARCHAR(50),
            atmosphere_rating NVARCHAR(50),
            service_type NVARCHAR(100),  
            meal_type NVARCHAR(100), 
            nation NVARCHAR(100),
            created_at DATETIME DEFAULT GETDATE(),
            CONSTRAINT FK_restaurant_review FOREIGN KEY (restaurant_id) REFERENCES restaurants (restaurant_id)
        )
        ''')

        conn.commit()
        return conn, cursor
    except Exception as e:
        print(f"Lỗi thiết lập cơ sở dữ liệu: {e}")
        raise

# Hàm hỗ trợ trích xuất dữ liệu
def safe_click(driver, element, wait_time=2):
    """Click an phần tử một cách an toàn bằng JavaScript và chờ đợi"""
    try:
        driver.execute_script("arguments[0].click();", element)
        time.sleep(wait_time)
        return True
    except Exception as e:
        print(f"Click thất bại: {e}")
        return False

def scroll_and_click_more(driver, scrollable_div, max_scrolls=20):
    """Cuộn và mở rộng nút 'Xem thêm' trong đánh giá"""
    last_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
    scroll_count = 0
    
    while scroll_count < max_scrolls:
        # Click tất cả nút "Xem thêm" đang hiển thị
        more_buttons = driver.find_elements(By.CSS_SELECTOR, "button.w8nwRe.kyuRq")
        for btn in more_buttons:
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", btn)
                time.sleep(0.5)
                btn.click()
            except:
                continue
        
        # Cuộn xuống
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
        time.sleep(1.5)
        
        # Kiểm tra dữ liệu mới
        new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
        if new_height == last_height:
            scroll_count += 1
            if scroll_count >= 3:  # Nếu không có chiều cao mới sau 3 lần, dừng lại
                break
        else:
            scroll_count = 0  # Đặt lại bộ đếm nếu tải nội dung mới
        
        last_height = new_height

# Hàm nhập dữ liệu chính
def import_scraped_data(driver, conn, cursor):
    """Nhập dữ liệu từ Google Maps vào cơ sở dữ liệu"""
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='feed']")))
        divs = element.find_elements(By.XPATH, "./div[position() > 2 and not(@class='TFQHme')]")
        links = []
        # Duyệt qua từng div để lấy link đầu tiên có trong đó
        for div in divs:
            a_tags = div.find_elements(By.CSS_SELECTOR, "a")  # Tìm tất cả thẻ <a> trong div
            if a_tags: 
                links.append(a_tags[0].get_attribute("href"))  # Lấy link đầu tiên trong div
        print(f"Tìm thấy {len(links)} nhà hàng để xử lý")
        
        for i, url in enumerate(reversed(links)):  
            try:
                # Kiểm tra độ dài URL
                if url and len(url) > 1999:
                    url = url[:1999]
                
                # Kiểm tra xem URL đã tồn tại trong cơ sở dữ liệu chưa
                cursor.execute("SELECT COUNT(*) FROM restaurants WHERE url = ?", (url,))
                exists = cursor.fetchone()[0] > 0
                
                if exists:
                    print(f"Nhà hàng {i+1}/{len(links)} đã tồn tại trong CSDL, bỏ qua.")
                    continue
                
                # Mở trang nhà hàng
                print(f"Đang mở nhà hàng {i+1}/{len(links)}")
                driver.get(url)
                # Trích xuất dữ liệu nhà hàng
                try:
                    restaurant_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[1]/h1'))).text
                except:
                    restaurant_name = None
                
                # Lấy mức giá
                try:
                    price = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/span/span/span/span[2]/span').text
                except:
                    price = None                       
                
                # Lấy đánh giá
                try:
                    rating = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]'))).text
                except:
                    rating = None
                
                # Lấy số điện thoại
                try:
                    # Danh sách các XPath có thể chứa số điện thoại
                    phone_xpaths = [
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[9]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[11]/div[10]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[13]/div[9]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[6]/button/div/div[2]/div[1]'
                    ]

                    phone = None  # Mặc định là None nếu không tìm thấy

                    for xpath in phone_xpaths:
                        try:
                            phone_elem = driver.find_element(By.XPATH, xpath)
                            phone = phone_elem.text.strip()
                            if phone:  # Nếu tìm thấy số điện thoại hợp lệ, thoát vòng lặp
                                break
                        except NoSuchElementException:
                            continue  # Nếu không tìm thấy, thử XPath tiếp theo

                except Exception as e:
                    print(f"Error getting phone number: {e}")
                
                # Lấy loại nhà hàng
                try:
                    type_elems = driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[2]/span[1]/span/button')
                    restaurant_type = ', '.join([elem.text for elem in type_elems])
                except:
                    restaurant_type = None
                
                # Lấy địa chỉ
                try:
                    address_xpaths = [
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[3]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[11]/div[3]/button/div/div[2]/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[13]/div[3]/button/div/div[2]/div[1]'
                    ]

                    address = None  

                    for xpath in address_xpaths:
                        try:
                            address_element = driver.find_element(By.XPATH, xpath)
                            address = address_element.text.strip()
                            if address:  # Nếu tìm thấy địa chỉ hợp lệ, thoát vòng lặp
                                break
                        except NoSuchElementException:
                            continue  # Nếu không tìm thấy, thử XPath tiếp theo

                except Exception as e:
                    print(f"Error getting address: {e}")

                # Lấy giờ mở cửa
                try:
                    expand_buttons = [
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[4]/div[1]/div[2]/div/span[2]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[13]/div[4]/button/div[1]',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[11]/div[4]/button'
                    ]

                    for xpath in expand_buttons:
                        try:
                            element = driver.find_element(By.XPATH, xpath)

                            # Cuộn từ từ xuống để tránh bị cuộn quá
                            for _ in range(5):  # Cuộn 5 lần, mỗi lần 100px
                                driver.execute_script("window.scrollBy(0, 100);")
                                time.sleep(0.2)

                            element.click()  # Click mở rộng nếu có
                            time.sleep(2)  # Đợi trang cập nhật dữ liệu
                            break  # Thoát vòng lặp nếu đã click thành công
                        except Exception:
                            continue  
                except Exception as e:
                    print(f"Error {e}")

                # Khởi tạo giá trị mặc định cho opening_hour
                opening_hour = None

                try:
                    opening_hours = [
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[2]/div/table',
                        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[4]/div[2]/div/table'
                    ]

                    for xpath in opening_hours:
                        try:
                            # Chờ bảng giờ mở cửa hiển thị sau khi mở rộng
                            element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath)))
                            opening_hour = element.text
                            if opening_hour:  # Nếu tìm thấy dữ liệu hợp lệ, thoát vòng lặp
                                break
                        except Exception:
                            continue  
                except Exception as e:
                    print(f"Error {e}")

                # Kiểm tra nếu opening_hour có dữ liệu hợp lệ trước khi xử lý
                if opening_hour:
                    lines = opening_hour.split("\n")
                    opening_hours_dict = {lines[i]: lines[i + 1] for i in range(0, len(lines), 2)}

                    # In kết quả
                    for day, hours in opening_hours_dict.items():
                        print(f"{day}: {hours}")
                    opening_hours_json = json.dumps(opening_hours_dict)
                else:
                    print("Không tìm thấy giờ mở cửa.")

                
                # Chèn dữ liệu nhà hàng
                try:
                    cursor.execute('''
                        INSERT INTO restaurants (url, restaurant_name, restaurant_type, rating, phone, price_level, address, opening_hours)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (url, restaurant_name, restaurant_type, rating, phone, price, address, opening_hours_json))
                    conn.commit()
                    
                    cursor.execute("SELECT IDENT_CURRENT('restaurants')")
                    restaurant_id = int(cursor.fetchone()[0])
                    print(f"Đã chèn nhà hàng với ID: {restaurant_id}")
                except Exception as e:
                    print(f"Lỗi khi chèn dữ liệu nhà hàng: {e}")
                    continue
                
                # Lấy đánh giá
                try:
                    reviews_tabs = driver.find_elements(By.XPATH, "//div[text()='Reviews']")
                    if not reviews_tabs:
                        print("Không tìm thấy tab đánh giá, bỏ qua đánh giá.")
                        continue
                        
                    safe_click(driver, reviews_tabs[0])
                    
                    try:
                        scrollable_divs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf")))
                        if not scrollable_divs:
                            continue
                            
                        scrollable_div = scrollable_divs[0]
                        scroll_and_click_more(driver, scrollable_div)
                    except TimeoutException:
                        print("  Hết thời gian chờ khi đợi phần đánh giá, bỏ qua đánh giá.")
                        continue
                    
                    review_containers = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf.fontBodyMedium")
                    print(f"  Tìm thấy {len(review_containers)} đánh giá")
                    
                    for container in review_containers:
                        try:
                            # Trích xuất thông tin đánh giá cơ bản
                            reviewer_name = container.find_element(By.CSS_SELECTOR, "div.d4r55").text
                            
                            try:
                                reviewer_info = container.find_element(By.CSS_SELECTOR, "div.RfnDt").text
                            except:
                                reviewer_info = None
                            
                            try:
                                rating_elem = container.find_element(By.CSS_SELECTOR, "span[aria-label]")
                                rating_text = rating_elem.get_attribute('aria-label').split()[0]
                                rating_value = float(rating_text.replace(',', '.'))
                            except:
                                rating_value = None
                            
                            try:
                                review_time = container.find_element(By.CSS_SELECTOR, "span.rsqaWe").text
                            except:
                                review_time = None
                            
                            try:
                                review_text = container.find_element(By.CSS_SELECTOR, "span.wiI7pd").text
                            except:
                                review_text = None
                            
                            # Trích xuất quốc gia (nếu có)
                            nation = None
                            try:
                                nation_elem = container.find_element(By.CSS_SELECTOR, "div.oqftme")
                                text = nation_elem.text
                                if "(" in text and ")" in text:
                                    nation = text.split("(")[-1].replace(")", "").strip()
                            except:
                                pass
                            
                            # Trích xuất đánh giá cụ thể
                            rating_elements = container.find_elements(By.CSS_SELECTOR, "span.RfDO5c")
                            service_rating = food_rating = atmosphere_rating = service_type = meal_type = None
                            
                            for elem in rating_elements:
                                text = elem.text.strip()
                                
                                if "Service:" in text:
                                    service_rating = text.split(":")[-1].strip()
                                elif "Food:" in text:
                                    food_rating = text.split(":")[-1].strip()
                                elif "Atmosphere:" in text:
                                    atmosphere_rating = text.split(":")[-1].strip()
                                elif text in ["Dine in", "Takeout"]:
                                    service_type = text
                                elif text in ["Breakfast", "Lunch", "Dessert", "Brunch", "Dinner", "Seating"]:
                                    meal_type = text
                            
                            # Chèn đánh giá này
                            cursor.execute('''
                                INSERT INTO reviews (restaurant_id, reviewer_name, reviewer_info, rating, review_time, 
                                            review_text, service_rating, food_rating, atmosphere_rating, 
                                            service_type, meal_type, nation)
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            ''', (restaurant_id, reviewer_name, reviewer_info, rating_value, review_time, 
                                  review_text, service_rating, food_rating, atmosphere_rating, 
                                  service_type, meal_type, nation))
                            
                        except Exception as e:
                            continue
                    
                    conn.commit()
                    print(f"  Đã nhập đánh giá thành công")
                    
                except Exception as e:
                    print(f"  Lỗi khi xử lý đánh giá cho nhà hàng {restaurant_name}: {e}")
                    conn.rollback()
                    
            except Exception as e:
                print(f"Lỗi khi xử lý nhà hàng ở vị trí {i}: {e}")
                try:
                    conn.rollback()
                except:
                    pass
    except Exception as e:
        print(f"Lỗi khi tìm liên kết nhà hàng: {e}")
    
    print("Hoàn thành nhập dữ liệu")

# Cấu hình trình duyệt
def setup_driver():
    """Thiết lập và cấu hình trình duyệt Chrome"""
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.implicitly_wait(10)
        
        # Giả lập vị trí ở Đà Nẵng
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", {
            "latitude": 16.0544,   # Vĩ độ Đà Nẵng
            "longitude": 108.2022, # Kinh độ Đà Nẵng
            "accuracy": 100
        })
        
        return driver
    except Exception as e:
        print(f"Lỗi khi thiết lập trình duyệt: {e}")
        raise

# Hàm cuộn xuống để lấy tất cả nhà hàng
def scroll_until_end(driver, scrollable_div, max_attempts=8):
    """Cuộn xuống cho đến khi tải hết tất cả nhà hàng"""
    last_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
    no_change_count = 0
    total_restaurants = 0
    last_restaurant_count = 0

    while no_change_count < max_attempts:
        restaurants = driver.find_elements(By.CSS_SELECTOR, ".Nv2PK.THOPZb.CpccDe")
        total_restaurants = len(restaurants)
        print(f"Tìm thấy {total_restaurants} nhà hàng (mới: {total_restaurants - last_restaurant_count})")
        
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
        time.sleep(3)

        new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)

        if new_height == last_height and total_restaurants == last_restaurant_count:
            no_change_count += 1
            print(f"Không tìm thấy dữ liệu mới: {no_change_count}/{max_attempts}")
        else:
            no_change_count = 0
            print(f"Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: {new_height}")
            
        last_height = new_height
        last_restaurant_count = total_restaurants
    
    print(f"Hoàn thành cuộn! Tìm thấy tổng cộng {total_restaurants} nhà hàng.")

# Thực thi chính
if __name__ == "__main__":
    driver = None
    conn = None
    
    try:
        print("Thiết lập kết nối cơ sở dữ liệu...")
        conn, cursor = setup_database()
        
        print("Khởi tạo trình duyệt...")
        driver = setup_driver()
        
        url = "https://www.google.com/maps/search/Restaurants+in+Da+Nang"
        print(f"Mở URL: {url}")
        driver.get(url)
        time.sleep(3)
        
        print("Tìm danh sách nhà hàng...")
        try:
            wait = WebDriverWait(driver, 20)
            scrollable_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='feed']")))
            
            print("Bắt đầu cuộn để tải thêm nhà hàng...")
            scroll_until_end(driver, scrollable_div)
            
            print("Bắt đầu nhập dữ liệu nhà hàng...")
            import_scraped_data(driver, conn, cursor)
            print("Nhập dữ liệu thành công!")
        except TimeoutException:
            print("Không thể tìm thấy phần tử danh sách nhà hàng sau 20 giây. Vui lòng kiểm tra bộ chọn hoặc kết nối.")
        
    except Exception as e:
        print(f"Lỗi trong quá trình thực thi: {e}")
    finally:
        if conn:
            try:
                conn.close()
                print("Đã đóng kết nối cơ sở dữ liệu")
            except:
                pass
        
        if driver:
            driver.quit()
            print("Đã đóng trình duyệt")

Thiết lập kết nối cơ sở dữ liệu...
Khởi tạo trình duyệt...
Mở URL: https://www.google.com/maps/search/Restaurants+in+Da+Nang
Tìm danh sách nhà hàng...
Bắt đầu cuộn để tải thêm nhà hàng...
Tìm thấy 6 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 2340
Tìm thấy 12 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 3333
Tìm thấy 18 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 3962
Tìm thấy 22 nhà hàng (mới: 4)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 4889
Tìm thấy 28 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 5862
Tìm thấy 34 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 6754
Tìm thấy 40 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 7052
Tìm thấy 42 nhà hàng (mới: 2)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 7987
Tìm thấy 48 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 8962
Tìm thấy 54 nhà hàng (mới: 6)
Đã cuộn và tìm thấy thêm dữ liệu! Chiều cao: 9968
Tìm thấy 60 n